In [1]:
!pip install surprise
import pandas as pd
import numpy as np

import surprise
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import warnings; warnings.simplefilter('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
rootdir = '/content/drive/My Drive/Colab Notebooks/Movie  Recommendation System/Movie Dataset/'

In [4]:
ratings = pd.read_csv(rootdir+'ratings.csv')

In [5]:
ratings.shape

(100836, 4)

In [7]:
movies = pd.read_csv(rootdir+'movies.csv')
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
print(ratings.shape)
ratings.head()

(100836, 4)


,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


**Data Preprocessing**

In [8]:
for dataset in [ratings,movies]:
  dataset.drop_duplicates(inplace=True)

In [10]:
ratings[ratings['userId'].isnull()]

,movieId,title,userId,rating


In [11]:
ratings[ratings['rating'].isnull()]

,movieId,title,userId,rating


**CF based recommendation system**

Our content based engine suffers from some severe limitations.

* It is only capable of suggesting movies which are close to a certain movie. That is, it is not capable of capturing tastes and providing recommendations across genres.
* Also, the engine that we built is not really personal in that it doesn't capture the personal tastes and biases of a user. Anyone querying our engine for recommendations based on a movie will receive the same recommendations for that movie, regardless of who (s)he is.
* Therefore, in this section, we will use Collaborative Filtering to make recommendations to Movie Watchers. Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will like a particular product or service those users have used/experienced but I have not.


**Item based**

In [12]:
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)

Before:  (610, 9719)
After:  (610, 2269)


In [13]:
userRatings

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),21 (2008),21 Grams (2003),21 Jump Street (2012),22 Jump Street (2014),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),30 Days of Night (2007),300 (2007),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",48 Hrs. (1982),50 First Dates (2004),...,"Wolf of Wall Street, The (2013)","Wolverine, The (2013)",Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988),Wonder Boys (2000),Wonder Woman (2017),Working Girl (1988),"World Is Not Enough, The (1999)",World War Z (2013),"World's End, The (2013)",Wreck-It Ralph (2012),"Wrestler, The (2008)",Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Apocalypse (2016),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),Yellow Submarine (1968),Yes Man (2008),Yojimbo (1961),You Don't Mess with the Zohan (2008),You Only Live Twice (1967),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [14]:
def standardize(row):
    new_row = (row - row.mean())/(row.max()-row.min())
    return new_row

* since each user has its own standard we can't assume A's rating 7 is same as another users' 7. So we use Correlation similarity. We use Shifted Cosine similarity also known as Pearson correlation

In [15]:
shiftedratings = userRatings.apply(standardize,axis=1).T
shiftedratings


userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",-0.081534,-0.009123,-0.003129,-0.056677,-0.014103,-0.072014,-0.039885,-0.014808,-0.009167,-0.030322,-0.020009,0.987616,-0.009343,-0.013839,-0.039841,-0.031512,-0.03896,-0.139489,0.273777,-0.066681,-0.10476,-0.026179,-0.03063,-0.038931,-0.009784,-0.007492,-0.038607,-0.113134,-0.02781,-0.014191,-0.015602,-0.032085,-0.042309,-0.022829,-0.008286,-0.012585,-0.007669,-0.022124,-0.034641,-0.031027,...,-0.014279,-0.050771,-0.107272,-0.008021,-0.006435,-0.005465,-0.046717,-0.004231,-0.024328,-0.121331,-0.015425,-0.019877,-0.014764,-0.030057,-0.022477,-0.067078,-0.04892,-0.015425,-0.014191,-0.19286,-0.014896,-0.028647,-0.029,-0.057162,-0.006699,-0.102688,-0.137329,-0.004231,0.057162,-0.164434,-0.036712,-0.037638,-0.172675,-0.026884,-0.047598,-0.204363,-0.05703,-0.208594,-0.009696,-0.257074
(500) Days of Summer (2009),-0.081534,-0.009123,-0.003129,-0.056677,-0.014103,-0.072014,-0.039885,-0.014808,-0.009167,-0.030322,-0.020009,-0.012384,-0.009343,-0.013839,0.760159,-0.031512,-0.03896,0.660511,-0.126223,-0.066681,-0.10476,0.073821,-0.03063,-0.038931,-0.009784,-0.007492,-0.038607,-0.113134,-0.02781,-0.014191,-0.015602,-0.032085,-0.042309,-0.022829,-0.008286,-0.012585,-0.007669,-0.022124,-0.034641,-0.031027,...,-0.014279,-0.050771,-0.107272,-0.008021,-0.006435,-0.005465,-0.046717,-0.004231,-0.024328,-0.121331,-0.015425,-0.019877,-0.014764,-0.030057,-0.022477,-0.067078,-0.04892,-0.015425,-0.014191,-0.19286,-0.014896,-0.028647,-0.029,-0.057162,-0.006699,-0.102688,-0.137329,-0.004231,0.157162,-0.164434,-0.036712,-0.037638,-0.172675,-0.026884,-0.047598,-0.204363,-0.05703,-0.208594,-0.009696,0.442926
10 Cloverfield Lane (2016),-0.081534,-0.009123,-0.003129,-0.056677,-0.014103,-0.072014,-0.039885,-0.014808,-0.009167,-0.030322,-0.020009,-0.012384,-0.009343,-0.013839,0.960159,-0.031512,-0.03896,-0.139489,-0.126223,-0.066681,-0.10476,-0.026179,-0.03063,-0.038931,-0.009784,-0.007492,-0.038607,-0.113134,-0.02781,-0.014191,-0.015602,-0.032085,-0.042309,-0.022829,-0.008286,-0.012585,-0.007669,-0.022124,-0.034641,-0.031027,...,-0.014279,-0.050771,-0.107272,-0.008021,-0.006435,-0.005465,-0.046717,-0.004231,-0.024328,-0.121331,-0.015425,-0.019877,-0.014764,-0.030057,-0.022477,-0.067078,-0.04892,-0.015425,-0.014191,-0.19286,-0.014896,-0.028647,-0.029,-0.057162,-0.006699,-0.102688,-0.137329,-0.004231,-0.342838,-0.164434,-0.036712,-0.037638,-0.172675,-0.026884,-0.047598,-0.204363,-0.05703,-0.208594,-0.009696,0.542926
10 Things I Hate About You (1999),-0.081534,-0.009123,-0.003129,-0.056677,-0.014103,-0.072014,-0.039885,-0.014808,-0.009167,-0.030322,-0.020009,0.987616,-0.009343,-0.013839,-0.039841,-0.031512,-0.03896,-0.139489,0.473777,-0.066681,-0.10476,-0.026179,-0.03063,-0.038931,-0.009784,-0.007492,-0.038607,-0.113134,-0.02781,-0.014191,-0.015602,-0.032085,-0.042309,-0.022829,-0.008286,-0.012585,-0.007669,-0.022124,-0.034641,-0.031027,...,-0.014279,-0.050771,-0.107272,-0.008021,0.393565,-0.005465,-0.046717,-0.004231,-0.024328,-0.121331,-0.015425,-0.019877,0.985236,-0.030057,-0.022477,-0.067078,-0.04892,-0.015425,-0.014191,-0.19286,-0.014896,-0.028647,-0.029,-0.057162,-0.006699,-0.102688,0.462671,-0.004231,0.257162,0.435566,-0.036712,-0.037638,0.427325,-0.026884,0.952402,-0.204363,-0.05703,-0.208594,-0.009696,-0.257074
"10,000 BC (2008)",-0.081534,-0.009123,-0.003129,-0.056677,-0.014103,-0.072014,-0.039885,-0.014808,-0.009167,-0.030322,-0.020009,-0.012384,-0.009343,-0.013839,-0.039841,-0.031512,-0.03896,-0.139489,-0.126223,-0.066681,-0.10476,-0.026179,-0.03063,-0.038931,-0.009784,-0.007492,-0.038607,-0.013134,-0.02781,-0.014191,-0.015602,-0.032085,-0.042309,-0.022829,-0.008286

In [16]:
cosine_sim = cosine_similarity(shiftedratings,shiftedratings)
cosine_sim

array([[ 1.        , -0.06122067,  0.03672852, ...,  0.12243083,
        -0.01455296,  0.3026466 ],
       [-0.06122067,  1.        ,  0.04584546, ..., -0.08492822,
         0.08467968, -0.01469542],
       [ 0.03672852,  0.04584546,  1.        , ...,  0.15434428,
         0.06650457, -0.02617503],
       ...,
       [ 0.12243083, -0.08492822,  0.15434428, ...,  1.        ,
         0.02843935,  0.02330033],
       [-0.01455296,  0.08467968,  0.06650457, ...,  0.02843935,
         1.        , -0.08878877],
       [ 0.3026466 , -0.01469542, -0.02617503, ...,  0.02330033,
        -0.08878877,  1.        ]])

In [17]:
resultDf = pd.DataFrame(cosine_sim,index=shiftedratings.index,columns=shiftedratings.index)

In [18]:
resultDf

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),21 (2008),21 Grams (2003),21 Jump Street (2012),22 Jump Street (2014),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),30 Days of Night (2007),300 (2007),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",48 Hrs. (1982),50 First Dates (2004),...,"Wolf of Wall Street, The (2013)","Wolverine, The (2013)",Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988),Wonder Boys (2000),Wonder Woman (2017),Working Girl (1988),"World Is Not Enough, The (1999)",World War Z (2013),"World's End, The (2013)",Wreck-It Ralph (2012),"Wrestler, The (2008)",Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Apocalypse (2016),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),Yellow Submarine (1968),Yes Man (2008),Yojimbo (1961),You Don't Mess with the Zohan (2008),You Only Live Twice (1967),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,-0.061221,0.036729,-0.001479,0.058118,0.000105,0.086202,-0.082214,0.057619,0.025464,0.043992,0.149754,0.024982,0.130444,0.119072,0.184711,0.137757,0.079141,0.215358,-0.077740,0.184217,-0.002787,-0.026386,-0.008998,-0.086366,-0.017249,-0.032319,0.034757,0.100613,-0.081331,-0.089329,0.148567,-0.159262,0.067957,-0.020195,0.082685,-0.099582,0.043871,0.243079,-0.112067,...,-0.104772,0.008986,0.080379,0.074964,0.091290,0.130018,-0.078230,-0.033651,-0.046300,-0.002188,-0.101299,0.105861,0.110477,-0.099772,-0.026012,0.049603,-0.110165,-0.103801,-0.118049,-0.165312,0.118840,-0.041708,0.022392,0.049310,0.042959,0.036848,-0.017407,0.187235,0.151339,0.286816,-0.092840,0.024116,0.114089,-0.036354,-0.091901,-0.182909,-0.051729,0.122431,-0.014553,0.302647
(500) Days of Summer (2009),-0.061221,1.000000,0.045845,0.179833,0.061246,0.045578,0.015461,0.080813,0.042163,0.110285,0.194225,-0.058642,0.073803,-0.090872,0.211679,-0.098252,-0.110270,0.086067,-0.086465,-0.033874,-0.097248,0.137214,0.185828,-0.032249,0.193010,0.098530,-0.092742,0.134370,-0.054113,0.018230,0.026268,0.074016,0.215128,-0.147711,0.108881,0.016565,0.236875,-0.070222,-0.014324,0.218176,...,0.169072,0.080904,-0.059211,-0.079470,0.081940,-0.096702,-0.123780,0.138137,0.164200,0.008419,0.276222,-0.078488,-0.095566,0.048067,0.179312,0.110676,0.194694,0.259691,0.123601,0.043233,-0.120798,0.341247,-0.106761,0.027900,-0.147370,0.012838,-0.077840,-0.080263,-0.083194,-0.096471,0.261026,0.070077,-0.065944,0.297938,0.252221,0.113452,0.138207,-0.084928,0.084680,-0.014695
10 Cloverfield Lane (2016),0.036729,0.045845,1.000000,-0.125633,0.205639,-0.052073,-0.135487,-0.061902,0.062900,0.304482,0.008806,-0.018396,0.114644,0.100099,0.103524,0.089015,0.081631,0.138270,0.104189,-0.156976,0.013272,0.234705,0.114388,-0.036048,0.284393,0.327149,-0.001100,0.070678,0.022284,0.054672,0.208629,0.308036,-0.058159,0.068474,-0.002428,0

In [25]:
def get_similar_movies(title):
    if title not in resultDf.index:
        return None
    else:
        similarList = zip(resultDf[title],list(resultDf.columns.values))
        similarList = sorted(similarList, key=lambda x: x[0], reverse=True)
        return similarList[1:10]
get_similar_movies('Captain America: The First Avenger (2011)')

[(0.6834659744265644, 'Thor (2011)'),
 (0.6297188643959462, 'Avengers: Age of Ultron (2015)'),
 (0.5724195140033145, 'Thor: The Dark World (2013)'),
 (0.5705625813597519, 'X-Men: Days of Future Past (2014)'),
 (0.5596586839539397, 'X-Men: First Class (2011)'),
 (0.5593651562822468, 'Iron Man 3 (2013)'),
 (0.510860518845639, 'Guardians of the Galaxy (2014)'),
 (0.5096435971380963, 'Avengers, The (2012)'),
 (0.5021585919704881, 'Star Trek Into Darkness (2013)')]